In [129]:
import numpy as np 
import torch

batch_size = 5


At first we have : 
- camera's parameters -> A (focal, center) 
- rotation matrix  -> R 
- position matrix -> C 
- 3D points position ->  P1 P2 P3 (and P4 to determinate the best solution after P3P)

In [130]:
# This script defines the camera parameters, rotation matrix, and translation matrix.
def camera() : 
  # Definition of the camera parameters
  # focal length
  fx = 800
  fy = 800
  # center
  cx = 320 
  cy = 240

  A = torch.tensor([[fx, 0, cx], [0, fy, cy], [0, 0, 1]], dtype=torch.float64) # intraseca matrix of the camera (3*3)
  #A = torch.from_numpy(A)  # Convert to a PyTorch tensor
  print("A = \n", A)
  print(A.shape)  # (3*3)
  A_batch = A.repeat(batch_size,1,1)
  print("A_batch = \n", A_batch)
  print(A_batch.shape)  # (batch_size, 3, 3)
  return A_batch

A = camera() 



def rotation_matrix() : 
  # Definition of the rotation matrix of the camera 
  R = torch.tensor([[1, 0, 0],[0, -1, 0], [0, 0, -1]], dtype=torch.float64)  # (3*3)
  #R = torch.from_numpy(R)  # Convert to a PyTorch tensor
  print("R = \n",R)
  print(R.shape)  # (3*3)
  R_batch = R.repeat(batch_size,1,1)  # Repeat the rotation matrix for each batch
  print("R_batch = \n", R_batch)  
  print(R_batch.shape)  # (batch_size, 3, 3)
  return R_batch

def camera_position() : 
  # Definition of the translation matrix of the camera (the position)
  C = torch.tensor([[0,0,6]], dtype=torch.float64)    # T = [tx,ty,tz]  (1*3)
  print("C = \n",C)
  print(C.shape)  # (1*3)

  C_batch = C.repeat(batch_size, 1)  # Repeat the translation vector for each batch
  print("C_batch = \n", C_batch)  
  print(C_batch.shape)  # (batch_size, 3)

 
  return C_batch

R = rotation_matrix()
C = camera_position()

A = 
 tensor([[800.,   0., 320.],
        [  0., 800., 240.],
        [  0.,   0.,   1.]], dtype=torch.float64)
torch.Size([3, 3])
A_batch = 
 tensor([[[800.,   0., 320.],
         [  0., 800., 240.],
         [  0.,   0.,   1.]],

        [[800.,   0., 320.],
         [  0., 800., 240.],
         [  0.,   0.,   1.]],

        [[800.,   0., 320.],
         [  0., 800., 240.],
         [  0.,   0.,   1.]],

        [[800.,   0., 320.],
         [  0., 800., 240.],
         [  0.,   0.,   1.]],

        [[800.,   0., 320.],
         [  0., 800., 240.],
         [  0.,   0.,   1.]]], dtype=torch.float64)
torch.Size([5, 3, 3])
R = 
 tensor([[ 1.,  0.,  0.],
        [ 0., -1.,  0.],
        [ 0.,  0., -1.]], dtype=torch.float64)
torch.Size([3, 3])
R_batch = 
 tensor([[[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.]],

        [[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.]],

        [[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.

In [131]:
#torch.manual_seed(1234)  

# Definition of 3D points in the world coordinate system
def point3Daleatoire(x) :
  # Generation of one random points in 3D space 
  return torch.empty((1, 3), dtype=torch.float64).uniform_(-x, x)


def pts_3D_4pts():
  # Generate randomly 4 3D points
  # Output : tensor which concatenate the 4 points = [ P1, P2, P3, P4 ] 

  P1 = point3Daleatoire(2)     # (1*3) -> pour P3P
  P2 = point3Daleatoire(2)
  P3 = point3Daleatoire(2)
  P4 = point3Daleatoire(2)
  
  points3D = torch.cat((P1,P2,P3,P4),dim=0);     # (LIGNES 4* COLONNES 3) - xyz
  print("points3D = \n", points3D)
  print(points3D.shape)  # (4*3)

  
  return points3D

def pts_3D_4pts_batch():
  # Generate randomly 4 3D points for each batch
  # Output : array which concatenate the 4 points = [ P1, P2, P3, P4 ] for each batch

  # Generate a batch of random points in 3D space
  # Each point is generated independently for each batch

  points3D_batch = torch.stack([pts_3D_4pts() for i in range(batch_size)])  # (batch_size, 4, 3)
  print("points3D_batch = \n", points3D_batch)
  print(points3D_batch.shape)  # (batch_size, 4, 3)
  return points3D_batch

points3D_batch = pts_3D_4pts_batch()  # Generate the batch of 3D points
'''
P1 = torch.tensor([0.7161, 0.5431, 1.7807], dtype=torch.float64)    # (3,)
P2 = torch.tensor([-1.1643, 0.8371, -1.0551], dtype=torch.float64)
P3 = torch.tensor([-1.5224, 0.4292, -0.1994], dtype=torch.float64)
P4 = torch.tensor([-1.5224, 0.4292, -0.1994], dtype=torch.float64) 
'''
P1 = points3D_batch[:, 0, :]  # Extract P1 for each batch
P2 = points3D_batch[:, 1, :]  # Extract P2 for each batch
P3 = points3D_batch[:, 2, :]  # Extract P3 for each batch
P4 = points3D_batch[:, 3, :]  # Extract P4 for each batch

print("P1 = \n", P1)
print("P2 = \n", P2)
print("P3 = \n", P3)
print("P4 = \n", P4)
print(P1.shape)  # (batch_size, 3)
print("batch_size = ", batch_size)


points3D = 
 tensor([[-0.5207,  1.0252, -1.4844],
        [-1.7653,  0.7157, -1.7130],
        [-1.7467, -1.1238,  1.9599],
        [ 1.3004,  1.5359,  0.6326]], dtype=torch.float64)
torch.Size([4, 3])
points3D = 
 tensor([[ 1.6953, -1.4334,  1.3427],
        [ 1.3867,  1.0870, -0.3620],
        [-1.7679, -0.1224, -0.4900],
        [ 0.0505,  1.3184,  1.4209]], dtype=torch.float64)
torch.Size([4, 3])
points3D = 
 tensor([[-0.5690, -0.9334, -0.7194],
        [ 1.6737,  0.5809,  1.7732],
        [ 1.5029, -1.5717,  0.9678],
        [-1.4868,  0.1351, -1.5274]], dtype=torch.float64)
torch.Size([4, 3])
points3D = 
 tensor([[-0.0642,  1.7793, -0.4778],
        [-1.2410, -1.8754,  0.3264],
        [ 0.3472, -0.1067, -0.2016],
        [-1.3920, -0.4856,  1.9540]], dtype=torch.float64)
torch.Size([4, 3])
points3D = 
 tensor([[ 0.3497, -1.3591, -1.2870],
        [-0.5084, -0.6820, -1.6066],
        [-1.4860,  1.9125, -1.2053],
        [-0.8128, -0.7862,  0.7045]], dtype=torch.float64)
torch.Siz

We create the 3 direction features vectors f1, f2, f3

In [132]:
def features_vectors(points3D,C, R,batch_size) :
    '''
    This function computes the features vectors for P3P algorithm.
    args:
    points3D : array with the 4 3D points = [ P1, P2, P3, P4 ] (4*3) 
    but we only use the first three points for P3P
    C: camera position matrix : (3*1)
    returns:
    featuresVect : array with the features vectors (9*1)
    '''
    P1 = torch.reshape(points3D[0], (batch_size,3,1))  # Reshape to (3,1) for easier calculations
    print("P1 = \n", P1)  # Print P1 to check the values
    print(P1.shape)  # (batch_size, 3, 1)
    P2 = torch.reshape(points3D[1], (batch_size,3,1))
    P3 = torch.reshape(points3D[2], (batch_size,3,1))

    C = torch.reshape(C, (batch_size,3,1))  # Reshape C to (3,1) for easier calculations
    print("C = \n", C)  # Print C to check the values
    print(C.shape)  # (batch_size, 3, 1)

    v1 = torch.matmul(R,(P1 - C))  # Calculate the vector from camera to P1
    print("v1 = \n", v1)  # Print v1 to check the values
    print(v1.shape)  # (batch_size, 3, 1)
    v2 = torch.matmul(R,(P2 - C))  # Calculate the vector from camera to P2
    v3 = torch.matmul(R,(P3 - C))  # Calculate the vector from camera to P3

    f1 = v1 / torch.norm(v1,dim=1, keepdim=True)  # Normalize the vector v1
    f2 = v2 / torch.norm(v2,dim=1, keepdim=True)  # Normalize the vector v2
    f3 = v3 / torch.norm(v3,dim=1, keepdim=True)  # Normalize the vector v3

    print("f1 = \n", f1)  # Print f1 to check the values
    print(f1.shape)  # (batch_size, 3, 1)

    f1 = torch.reshape(f1, (batch_size,1,3))  # Reshape to (3,1)
    print("f1 : ",f1.shape) # (batch_size,1,3)
    f2 = torch.reshape(f2, (batch_size,1,3))
    f3 = torch.reshape(f3, (batch_size,1,3))

    featuresVect = torch.cat((f1,f2,f3),dim=1)
    print("features vectors = \n",featuresVect)
    print(featuresVect.shape)  # (batch_size, 3, 3)

    return featuresVect # Return the features vectors need in P3P


points3D = [P1, P2, P3]  # We define the points3D with the first three points
print("points3D = \n", points3D)  # Print the points3D to check the values  / List len = 3 

print("batch_size = ", batch_size)
featuresVect = features_vectors(points3D, C, R,batch_size)  

points3D = 
 [tensor([[-0.5207,  1.0252, -1.4844],
        [ 1.6953, -1.4334,  1.3427],
        [-0.5690, -0.9334, -0.7194],
        [-0.0642,  1.7793, -0.4778],
        [ 0.3497, -1.3591, -1.2870]], dtype=torch.float64), tensor([[-1.7653,  0.7157, -1.7130],
        [ 1.3867,  1.0870, -0.3620],
        [ 1.6737,  0.5809,  1.7732],
        [-1.2410, -1.8754,  0.3264],
        [-0.5084, -0.6820, -1.6066]], dtype=torch.float64), tensor([[-1.7467, -1.1238,  1.9599],
        [-1.7679, -0.1224, -0.4900],
        [ 1.5029, -1.5717,  0.9678],
        [ 0.3472, -0.1067, -0.2016],
        [-1.4860,  1.9125, -1.2053]], dtype=torch.float64)]
batch_size =  5
P1 = 
 tensor([[[-0.5207],
         [ 1.0252],
         [-1.4844]],

        [[ 1.6953],
         [-1.4334],
         [ 1.3427]],

        [[-0.5690],
         [-0.9334],
         [-0.7194]],

        [[-0.0642],
         [ 1.7793],
         [-0.4778]],

        [[ 0.3497],
         [-1.3591],
         [-1.2870]]], dtype=torch.float64)
torch.Si

Lastly we need the functions to resolve the polynomial roots. - for test go to test resolution polynome 

In [133]:
from complex_batch_utils import *
import complex_utils as cpu
from torch import vmap

def polynomial_root_calculation_3rd_degree(a, b, c, d):
    # This function calculates the roots of a cubic polynomial of the form:
    # a*x^3 + b*x^2 + c*x + d = 0
        # a (batch_size, 1)
        # b (batch_size, 1)
        # c (batch_size, 1)
        # d (batch_size, 1)

    # output: roots of the polynomial in the form of a tensor of shape (batch_size,3, 2)
    # where each root is represented as a complex number (real, imaginary)
    # each row is the i_th root of the polynomial
    
    batch_size = a.shape[0]  # Get the batch size from the shape of a
    

    # Discriminant terms
    p = (3 * a * c - b**2) / (3 * a**2)     # (batch_size, 1) because element-wise opeations
    q = (2 * b**3 - 9 * a * b * c + 27 * a**2 * d) / (27 * a**3)    # (batch_size, 1)
    delta = -4 * p**3 - 27 * q**2   # (batch_size, 1)

    roots = torch.empty((batch_size,3, 2))  # Initialize roots tensor to store the roots

    j_ = torch.tensor([-0.5, torch.sqrt(torch.tensor(3))/2])  # cube root of unity
    
    for k in range (3):
        delta_sur_27 = -delta / 27   #(batch_size, 1) 

        sqrt_term = sqrt_batch(delta_sur_27)  

        # faire une seule fois les calculs de j^k et j^-k
        j_exp_k = cpu.complex_number_power_k(j_, k)  # Compute j^k for each batch
        j_exp_moins_k = cpu.complex_number_power_k(j_, -k)  # Compute j^-k for each batch

        j_exp_k_batch = j_exp_k.repeat(batch_size, 1)
        j_exp_moins_k_batch = j_exp_moins_k.repeat(batch_size, 1)

        u_k = product_of_2_complex_numbers_batch(j_exp_k_batch, sqrt_3_batch(torch.stack([0.5*(-q.squeeze()+sqrt_term[:,0]),sqrt_term[:,1]],dim=-1)) )
         # (batch_size, 2) 
        v_k = product_of_2_complex_numbers_batch(j_exp_moins_k_batch, sqrt_3_batch(torch.stack([0.5*(-q.squeeze()-sqrt_term[:,0]),-0.5*sqrt_term[:,1]],dim=-1)))
          # (batch_size, 2) 

        root = addition_batch(addition_batch(u_k, v_k), torch.stack([-b[:,0]/(3*a[:,0]),0.0*b[:,0]],dim=-1) ) 
         # (batch_size, 2)

        roots[:,k,:] = root  # Store the root in the roots tensor

    return roots


In [134]:

def polynomial_root_calculation_4th_degree_ferrari(a0, a1, a2, a3, a4): # Ferrari's Method
    print("a0 = \n", a0)  # Print a0 to check the values
    print(a0.shape)  # (batch_size, 1)

    
    # Solving a polynomial of 4th degree
    # a0, a1, a2, a3, a4 (batch_size,1)

    # Output : roots of the polynomial a4*x^4 + a3*x^3 + a2*x^2 + a1*x + a0   (4,batch_size 2) !! ATTENTION !!! 

    batch_size = a0.shape[0]  # Get the batch size from the shape of a0

    # Reduce the quartic equation to the form : x^4 + a*x^3 + b*x^2 + c*x + d = 0
    a = a3/a4           # (batch_size, 1)
    print("a",a.shape)  # (batch_size, 1)
    b = a2/a4
    c = a1/a4
    d = a0/a4

    # Computation of the coefficients of the Ferrari's Method
    S = a/4     # (batch_size, 1)
    b0 = d - c*S + b* S**2 - 3* S**4    # (batch_size, 1)
    b1 = c - 2*b*S + 8*S**3 # (batch_size, 1)
    b2 = b - 6 * S**2  # (batch_size, 1)


    # Solve the cubic equation m^3 + b2*m^2 + (b2^2/4  - b0)*m - b1^2/8 = 0
    x_cube = polynomial_root_calculation_3rd_degree(torch.tensor(1).repeat(b2.shape),b2,(b2**2)/4-b0,(-b1**2)/8)
    print("x_cube ", x_cube)  # Print the roots of the cubic equation
    print(x_cube.shape)
    

    x_cube_real = x_cube[:,:,0]  # Extract the real part of the roots  (batch_size, 3)
    print("x_cube_real = \n", x_cube_real)  
    print(x_cube_real.shape)  

    x_cube_imag = x_cube[:,:,1]  # Extract the imaginary part of the roots    (batch_size, 3)

    is_real = torch.isclose(x_cube_imag,torch.tensor(0.0))
    is_positive = x_cube[:,:,0] > 0
    condition = is_real & is_positive  # Condition to check if the root is real and positive   (batch_size, 3)
   
    
    real_filtered = x_cube_real.clone()
    real_filtered[~condition] = float('inf')  # if root real and positive, keep it, else set to infinity   (batch_size, 3)


    alpha_0_real, _ = real_filtered.min(dim=1) # Get the minimum real part of the roots (if doesn't exist, returns inf)

    alpha_0 = torch.stack([alpha_0_real, torch.zeros(batch_size)], dim=-1)  # (batch_size,2)
    

    # do the calculation for alpha_0_nul and not alpha_0_nul and then affects the good value 

    # if alpha_0_nul == False
    alpha0_div_2 = 0.5*alpha_0              # beacause alpha_0 is real  # (batch_size, 2)    
    sqrt_alpha = sqrt_batch(alpha0_div_2[:,0].unsqueeze(-1))    # input : (batch_size, 1) // output : (batch_size, 2)
    term = addition_complex_real_batch(- alpha0_div_2 ,-b2 / 2)      # (batch_size, 2)
    denom = 2 * torch.sqrt(2 * alpha_0)      # beacause alpha_0 is real  # (batch_size, 2)
    num = torch.stack([b1, torch.zeros(batch_size,1)], dim=-1).squeeze(1)  # (batch_size, 2)
   

    frac = division_2_complex_numbers(num,denom)    # (batch_size, 2)
    print("frac = \n", frac)  # Print the fraction to check the values
    print(frac.shape)  # (batch_size, 2)

    x1_false = addition_complex_real_batch(sqrt_alpha ,- S) + sqrt_complex_batch(addition_batch(term,-frac))    # (batch_size, 2)
    x2_false = addition_complex_real_batch(sqrt_alpha, - S) - sqrt_complex_batch(addition_batch(term,-frac))    # (batch_size, 2)
    x3_false = addition_complex_real_batch(-sqrt_alpha, - S) + sqrt_complex_batch(addition_batch(term,frac))    # (batch_size, 2)
    x4_false = addition_complex_real_batch(-sqrt_alpha,- S) - sqrt_complex_batch(addition_batch(term,frac))     # (batch_size, 2)
    
    # if alpha_0_nul == True
    print("b2 ", b2.shape)  # (batch_size, 1)
    print((b2**2) / 4 - b0)
    sqrt_inner1 = sqrt_batch((b2**2) / 4 - b0)        # complex 

    x1_true = addition_complex_real_batch(sqrt_complex_batch(addition_complex_real_batch(sqrt_inner1,-b2 / 2)),-S)
    x2_true = addition_complex_real_batch(- sqrt_complex_batch(addition_complex_real_batch(sqrt_inner1,-b2 / 2)),-S)
    x3_true = addition_complex_real_batch(sqrt_complex_batch(addition_complex_real_batch(- sqrt_inner1,-b2 / 2 )),-S)
    x4_true = addition_complex_real_batch(- sqrt_complex_batch(addition_complex_real_batch(- sqrt_inner1,-b2 / 2 )),-S)
    
    result = torch.where(alpha_0==float('inf'),
                         torch.stack([x1_true, x2_true, x3_true, x4_true]),torch.stack([x1_false, x2_false, x3_false, x4_false]))
    print("result = \n", result.shape)  # Print the result to check the values
    return result   # (4,batch_size, 2)

print("batch_size = ", batch_size) 
a0  = torch.rand(batch_size, 1, dtype=torch.float64)
a1  = torch.rand(batch_size, 1, dtype=torch.float64)  
a2  = torch.rand(batch_size, 1, dtype=torch.float64)  
a3  = torch.rand(batch_size, 1, dtype=torch.float64) 
a4  = torch.rand(batch_size, 1, dtype=torch.float64)  
print(polynomial_root_calculation_4th_degree_ferrari(a0, a1, a2, a3, a4))  # Call the function to calculate the roots of the polynomial


batch_size =  5
a0 = 
 tensor([[0.0658],
        [0.5262],
        [0.9229],
        [0.3747],
        [0.8758]], dtype=torch.float64)
torch.Size([5, 1])
a torch.Size([5, 1])
x_cube  tensor([[[ 0.4666,  0.0000],
         [ 0.0823,  0.5481],
         [ 0.0823, -0.5481]],

        [[ 0.8265,  0.0000],
         [-0.3769,  0.1512],
         [-0.3769, -0.1512]],

        [[-1.2021,  0.0000],
         [ 0.3809,  0.0000],
         [ 0.3809,  0.0000]],

        [[ 1.8487,  0.0000],
         [-0.0208,  0.4416],
         [-0.0208, -0.4416]],

        [[-2.0251,  0.0000],
         [ 0.1330,  0.0000],
         [ 0.1330,  0.0000]]])
torch.Size([5, 3, 2])
x_cube_real = 
 tensor([[ 0.4666,  0.0823,  0.0823],
        [ 0.8265, -0.3769, -0.3769],
        [-1.2021,  0.3809,  0.3809],
        [ 1.8487, -0.0208, -0.0208],
        [-2.0251,  0.1330,  0.1330]])
torch.Size([5, 3])
a.shape =  torch.Size([5, 2])
b.shape =  torch.Size([5, 1])
frac = 
 tensor([[ 0.5543,  0.0000],
        [ 0.4061,  0.0000],
    

We have all the variables needed for the p3p so we start

1. Storage of points : already done 

In [135]:
print("P1 = \n", P1)
print("batch_size = ", batch_size)
print("P2 = \n", P2)
print("P3 = \n", P3)

P1 = 
 tensor([[-0.5207,  1.0252, -1.4844],
        [ 1.6953, -1.4334,  1.3427],
        [-0.5690, -0.9334, -0.7194],
        [-0.0642,  1.7793, -0.4778],
        [ 0.3497, -1.3591, -1.2870]], dtype=torch.float64)
batch_size =  5
P2 = 
 tensor([[-1.7653,  0.7157, -1.7130],
        [ 1.3867,  1.0870, -0.3620],
        [ 1.6737,  0.5809,  1.7732],
        [-1.2410, -1.8754,  0.3264],
        [-0.5084, -0.6820, -1.6066]], dtype=torch.float64)
P3 = 
 tensor([[-1.7467, -1.1238,  1.9599],
        [-1.7679, -0.1224, -0.4900],
        [ 1.5029, -1.5717,  0.9678],
        [ 0.3472, -0.1067, -0.2016],
        [-1.4860,  1.9125, -1.2053]], dtype=torch.float64)


2. Storage of the features vectors : done

In [136]:
# we got featuresVect and we access the 3 values 
f1 = featuresVect[:,0,:]  # Access the first feature vector for each batch
f2 = featuresVect[:,1,:]
f3 = featuresVect[:,2,:]

print("f1 = ", f1)
print(f1.shape)  # (batsh_size,3)
print("f2 = ", f2)
print(f2.shape)  # (batsh_size,3)
print("f3 = ", f3)
print(f3.shape)  # (batsh_size,3)

f1 =  tensor([[-0.0688, -0.1354,  0.9884],
        [ 0.3286,  0.2778,  0.9027],
        [-0.0836,  0.1371,  0.9870],
        [-0.0096, -0.2649,  0.9642],
        [ 0.0471,  0.1831,  0.9820]], dtype=torch.float64)
torch.Size([5, 3])
f2 =  tensor([[-0.2222, -0.0901,  0.9708],
        [ 0.2101, -0.1647,  0.9637],
        [ 0.3652, -0.1268,  0.9223],
        [-0.2033,  0.3073,  0.9296],
        [-0.0664,  0.0891,  0.9938]], dtype=torch.float64)
torch.Size([5, 3])
f3 =  tensor([[-0.3845,  0.2474,  0.8894],
        [-0.2628,  0.0182,  0.9647],
        [ 0.2742,  0.2867,  0.9180],
        [ 0.0559,  0.0172,  0.9983],
        [-0.1955, -0.2516,  0.9479]], dtype=torch.float64)
torch.Size([5, 3])


3. Création of a solution variable : maximum 4 solutions  

    Matrix (4,3,4)  
    Each layer is a solution, for each leayer : first column stres the camera position matrix C (3,1) and the remaining 3 columns store the rotation matrix R (3,3)

In [137]:
solutions = torch.empty((batch_size,4,3,4), dtype=torch.float64)
print("batch_size = ", batch_size)
print("solutions = \n", solutions)
print(solutions.shape)  # (batch_size,4,3,4)

batch_size =  5
solutions = 
 tensor([[[[1.5636e-311, 4.9407e-324, 4.9407e-324, 4.9407e-324],
          [4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324],
          [4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324]],

         [[4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324],
          [4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324],
          [4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324]],

         [[4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324],
          [4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324],
          [4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324]],

         [[4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324],
          [4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324],
          [4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324]]],


        [[[4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324],
          [4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324],
          [4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324]],

4. Verification that the 3 points given are not collinear 

In [138]:
# Test of non-collinearity
print("P1 = \n", P1)
print(P1.shape)  # (batch_size, 3)

v1 = P2 - P1
print("v1 = \n", v1)  # Print v1 to check the values
print(v1.shape)  # (batch_size, 3)
v2 = P3 - P1
print("v2 = \n", v2)  # Print v2 to check the values
norms = torch.norm(torch.cross(v1,v2, dim=1),dim = 1 )
print("norms : ",norms)  # (batch_size,)

all_dif_zero = torch.all(norms != 0)  # Check if all norms are non-zero

if not all_dif_zero:
    print('\nProblem: the points must not be collinear')
else:
    print('\nThe points are not collinear, we can continue')

P1 = 
 tensor([[-0.5207,  1.0252, -1.4844],
        [ 1.6953, -1.4334,  1.3427],
        [-0.5690, -0.9334, -0.7194],
        [-0.0642,  1.7793, -0.4778],
        [ 0.3497, -1.3591, -1.2870]], dtype=torch.float64)
torch.Size([5, 3])
v1 = 
 tensor([[-1.2446, -0.3096, -0.2286],
        [-0.3087,  2.5204, -1.7047],
        [ 2.2426,  1.5143,  2.4926],
        [-1.1768, -3.6547,  0.8041],
        [-0.8581,  0.6770, -0.3195]], dtype=torch.float64)
torch.Size([5, 3])
v2 = 
 tensor([[-1.2260, -2.1490,  3.4443],
        [-3.4632,  1.3110, -1.8327],
        [ 2.0719, -0.6383,  1.6872],
        [ 0.4114, -1.8860,  0.2762],
        [-1.8357,  3.2716,  0.0818]], dtype=torch.float64)
norms :  tensor([ 5.3433, 10.1721,  6.3223,  3.8142,  2.0224], dtype=torch.float64)

The points are not collinear, we can continue


5. Creation of an orthonormal frame from f1, f2, f3 (the features vectors)

In [139]:
# Calculation of vectors of the base τ = (C,tx,ty,tz)
tx = f1     # (batch_size,3)
print("tx = ", tx)
print(tx.shape)  # (batch_size,3)

tz = torch.cross(f1,f2,dim=1)/ torch.norm(torch.cross(f1,f2,dim=1),dim=1, keepdim=True) 
# (batch_size,3)
print("tz = ", tz)
print(tz.shape)  # (batch_size,3)

ty = torch.cross(tz,tx,dim=1) # (batch_size,3)
print("ty = ", ty)
print(ty.shape)  # (batch_size,3)

tx =  tensor([[-0.0688, -0.1354,  0.9884],
        [ 0.3286,  0.2778,  0.9027],
        [-0.0836,  0.1371,  0.9870],
        [-0.0096, -0.2649,  0.9642],
        [ 0.0471,  0.1831,  0.9820]], dtype=torch.float64)
torch.Size([5, 3])
tz =  tensor([[-0.2644, -0.9529, -0.1489],
        [ 0.9261, -0.2826, -0.2501],
        [ 0.4969,  0.8643, -0.0780],
        [-0.9407, -0.3246, -0.0985],
        [ 0.6407, -0.7597,  0.1109]], dtype=torch.float64)
torch.Size([5, 3])
ty =  tensor([[-0.9620,  0.2715, -0.0297],
        [-0.1856, -0.9181,  0.3501],
        [ 0.8638, -0.4840,  0.1404],
        [-0.3391,  0.9080,  0.2461],
        [-0.7663, -0.6239,  0.1531]], dtype=torch.float64)
torch.Size([5, 3])


5. (bis) Creation of a transformation matrix T and expression of the f3 vector in this frame

In [140]:
tx = torch.reshape(tx,(batch_size,1,3))   # (batch_size,1,3)
print("tx = \n", tx)

ty = torch.reshape(ty,(batch_size,1,3))  # (batch_size,1,3)
print("ty = \n", ty)

tz = torch.reshape(tz,(batch_size,1,3))  # (batch_size,1,3)
print("tz = \n", tz)

# Computation of the matrix T and the feature vector f3
T = torch.cat((tx,ty,tz),dim = 1) # (3*3)
print("T = \n", T)
print(T.shape)  # (batch_size,3,3)


f3_T = torch.matmul(T,f3.unsqueeze(-1)) # (
print("f3_T = \n", f3_T)
print(f3_T.shape)  # (batch_size,3,1)


tx = 
 tensor([[[-0.0688, -0.1354,  0.9884]],

        [[ 0.3286,  0.2778,  0.9027]],

        [[-0.0836,  0.1371,  0.9870]],

        [[-0.0096, -0.2649,  0.9642]],

        [[ 0.0471,  0.1831,  0.9820]]], dtype=torch.float64)
ty = 
 tensor([[[-0.9620,  0.2715, -0.0297]],

        [[-0.1856, -0.9181,  0.3501]],

        [[ 0.8638, -0.4840,  0.1404]],

        [[-0.3391,  0.9080,  0.2461]],

        [[-0.7663, -0.6239,  0.1531]]], dtype=torch.float64)
tz = 
 tensor([[[-0.2644, -0.9529, -0.1489]],

        [[ 0.9261, -0.2826, -0.2501]],

        [[ 0.4969,  0.8643, -0.0780]],

        [[-0.9407, -0.3246, -0.0985]],

        [[ 0.6407, -0.7597,  0.1109]]], dtype=torch.float64)
T = 
 tensor([[[-0.0688, -0.1354,  0.9884],
         [-0.9620,  0.2715, -0.0297],
         [-0.2644, -0.9529, -0.1489]],

        [[ 0.3286,  0.2778,  0.9027],
         [-0.1856, -0.9181,  0.3501],
         [ 0.9261, -0.2826, -0.2501]],

        [[-0.0836,  0.1371,  0.9870],
         [ 0.8638, -0.4840,  0.1404],
  

The sing of the z-coordinate in f3_T give us the sign of teta, that we will need after 

In [141]:
print(f3_T[:,2])
f3_T_positif = f3_T[:,2] > 0

print("f3_T_positif = \n", f3_T_positif)
print(f3_T_positif.shape)  # (batch_size,1)

tensor([[-0.2665],
        [-0.4898],
        [ 0.3124],
        [-0.1565],
        [ 0.1710]], dtype=torch.float64)
f3_T_positif = 
 tensor([[False],
        [False],
        [ True],
        [False],
        [ True]])
torch.Size([5, 1])


6. Change of frame is performed on the 3D points side, and the transformation matrix N is defined

In [142]:
# Calculation of vectors of the base η = (P1,nx,ny,nz)
nx = (P2 - P1)/torch.norm(P2 - P1,dim=1,keepdim=True)      #(batch_size,3)
print("nx : ", nx.shape)  # (batch_size,3)

nz = torch.cross(nx,P3-P1,dim=1)/torch.norm(torch.cross(nx,P3-P1,dim=1), dim=1, keepdim=True) 
print("nz : ", nz.shape)  # (batch_size,3)

ny = torch.cross(nz,nx,dim=1)
print("ny : ", ny.shape)  # (batch_size,3)


# Reshape the vectors to (1,3) for concatenation
nx = torch.reshape(nx,(batch_size,1,3))  # (batch_size,1,3)
ny = torch.reshape(ny,(batch_size,1,3))
nz = torch.reshape(nz,(batch_size,1,3))

print("nx = \n", nx)
print(nx.shape)  # (1*3)
print("ny = \n", ny)
print("nz = \n", nz)

# Computation of the matrix N and the world point P3
N = torch.cat((nx,ny,nz),dim = 1) #  T's equivalent in the world coordinate system
print("N = \n", N)
print(N.shape)  # (batch_size,3,3)

print("P3.shape = ", P3.shape)  # (batch_size,3)

P3_n = torch.matmul(N,(P3-P1).unsqueeze(-1)) 


print("P3_n = \n", P3_n)
print(P3_n.shape)  # (5,3,1)


nx :  torch.Size([5, 3])
nz :  torch.Size([5, 3])
ny :  torch.Size([5, 3])
nx = 
 tensor([[[-0.9554, -0.2376, -0.1755]],

        [[-0.1009,  0.8241, -0.5574]],

        [[ 0.6096,  0.4116,  0.6775]],

        [[-0.3000, -0.9317,  0.2050]],

        [[-0.7535,  0.5945, -0.2806]]], dtype=torch.float64)
torch.Size([5, 1, 3])
ny = 
 tensor([[[-0.0479, -0.4615,  0.8858]],

        [[-0.9669, -0.2132, -0.1402]],

        [[ 0.4454, -0.8848,  0.1368]],

        [[ 0.9446, -0.3201, -0.0723]],

        [[ 0.3688,  0.7356,  0.5682]]], dtype=torch.float64)
nz = 
 tensor([[[-0.2915,  0.8547,  0.4295]],

        [[-0.2344,  0.5248,  0.8183]],

        [[ 0.6558,  0.2184, -0.7227]],

        [[ 0.1330,  0.1719,  0.9761]],

        [[ 0.5442,  0.3247, -0.7735]]], dtype=torch.float64)
N = 
 tensor([[[-0.9554, -0.2376, -0.1755],
         [-0.0479, -0.4615,  0.8858],
         [-0.2915,  0.8547,  0.4295]],

        [[-0.1009,  0.8241, -0.5574],
         [-0.9669, -0.2132, -0.1402],
         [-0.2344,  0

7. Definition of the variables for the following steps 

In [143]:
# Computation of phi1 et phi2 with 0=x, 1=y, 2=z
phi1 = f3_T[:,0]/f3_T[:,2]
phi2 = f3_T[:,1]/f3_T[:,2]
print("phi1 = ", phi1)
print(phi1.shape)  # (batch_size,1)
print("phi2 = ", phi2)
print(phi2.shape)  # (batch_size,1)

# Extraction of p1 and p2 from P3_eta
p1 = P3_n[:,0] #x
p2 = P3_n[:,1] #y
print("p1 = ", p1)
print(p1.shape)  # (batch_size,3)
print("p2 = ", p2)
print(p2.shape)  # (batch_size,3)

# Computation of d12
d12 = torch.norm(P2-P1,dim=1, keepdim=True) 
print("d12 = ", d12)
print(d12.shape)  # (batch_size,1)

# Computation of b = cot(beta)
cosBeta =( torch.sum(f1*f2,dim=1)/(torch.norm(f1,dim=1)*torch.norm(f2,dim=1)) ).unsqueeze(-1)  # tensor.dot(a,b) <=> tensor.sum(a*b)
print("cosBeta = ", cosBeta)  
print(cosBeta.shape)  # (batch_size,1)

b = torch.sqrt(1/(1-cosBeta**2)-1)

b = torch.where(cosBeta < 0, -b, b)  # If cosBeta < 0, then b = -b
print("b = ", b)
print(b.shape)  # (batch_size,1)

phi1 =  tensor([[-3.2718],
        [-1.6120],
        [ 2.9524],
        [-6.1198],
        [ 5.1186]], dtype=torch.float64)
torch.Size([5, 1])
phi2 =  tensor([[-1.5407],
        [-0.7551],
        [ 0.7262],
        [-1.5485],
        [ 2.6423]], dtype=torch.float64)
torch.Size([5, 1])
p1 =  tensor([[1.0775],
        [2.4514],
        [2.1433],
        [1.6903],
        [3.3054]], dtype=torch.float64)
torch.Size([5, 1])
p2 =  tensor([[4.1017],
        [3.3259],
        [1.7184],
        [0.9723],
        [1.7760]], dtype=torch.float64)
torch.Size([5, 1])
d12 =  tensor([[1.3027],
        [3.0584],
        [3.6791],
        [3.9228],
        [1.1388]], dtype=torch.float64)
torch.Size([5, 1])
cosBeta =  tensor([[0.9870],
        [0.8932],
        [0.8624],
        [0.8170],
        [0.9891]], dtype=torch.float64)
torch.Size([5, 1])
b =  tensor([[6.1529],
        [1.9865],
        [1.7036],
        [1.4166],
        [6.7056]], dtype=torch.float64)
torch.Size([5, 1])


8. Calculation of the coefficients of the polynomial 

In [144]:
a4 = - phi2**2 * p2**4 - phi1**2 * p2**4 - p2**4
a3 = 2 * p2**3 * d12 * b + 2 * phi2**2 * p2**3 * d12 * b - 2 * phi1 * phi2 * p2**3 * d12
a2 = - phi2**2 * p1**2 * p2**2 - phi2**2 * p2**2 * d12**2 * b**2 - phi2**2 * p2**2 * d12**2 + phi2**2 * p2**4 + phi1**2 * p2 **4 + 2 * p1 * p2**2 * d12 + 2 * phi1 * phi2 * p1 * p2**2 * d12 * b - phi1**2 * p1**2 * p2**2 + 2 * phi2**2 * p1 * p2**2 * d12 - p2**2 * d12**2 * b**2 - 2 * p1**2 * p2**2
a1 = 2 * p1**2 * p2 * d12 * b + 2 * phi1 * phi2 * p2**3 * d12 - 2 * phi2**2 * p2**3 * d12 * b - 2 * p1 * p2 * d12**2 * b
a0 = - 2 * phi1 * phi2 * p1 * p2**2 * d12 * b + phi2**2 * p2**2 * d12**2 + 2 * p1**3 * d12 - p1**2 * d12**2 + phi2**2 * p1**2 * p2**2 - p1**4 - 2 * phi2**2 * p1 * p2**2 * d12 + phi1**2 * p1**2 * p2**2 + phi2**2 * p2**2 * d12**2 * b**2

print("a4 = ", a4)
print(a4.shape)  # (batch_size,1)
print("a3 = ", a3)
print("a3.shape = ", a3.shape)  # (batch_size,1)
print("a2 = ", a2)
print(a2.shape)  # (batch_size,1)
print("a1 = ", a1)
print(a1.shape)  # (batch_size,1)
print("a0 = ", a0)
print(a0.shape)  # (batch_size,1)


a4 =  tensor([[-3984.6414],
        [ -510.0890],
        [  -89.3258],
        [  -36.5110],
        [ -340.0491]], dtype=torch.float64)
torch.Size([5, 1])
a3 =  tensor([[2825.7003],
        [ 428.0347],
        [  17.0990],
        [ -33.6305],
        [ 510.2995]], dtype=torch.float64)
a3.shape =  torch.Size([5, 1])
a2 =  tensor([[ 1.3170e+03],
        [ 5.5163e+00],
        [-2.8943e+01],
        [-6.0026e-01],
        [-3.5858e+01]], dtype=torch.float64)
torch.Size([5, 1])
a1 =  tensor([[-1735.4518],
        [  -41.1138],
        [  -24.3958],
        [    3.0683],
        [ -230.5091]], dtype=torch.float64)
torch.Size([5, 1])
a0 =  tensor([[ 1.3118e+03],
        [ 4.5853e+00],
        [ 2.1543e+00],
        [-8.1239e-02],
        [ 8.5578e+01]], dtype=torch.float64)
torch.Size([5, 1])


9. Recovery of the polynomial roots cos (teta)

In [145]:
from torch import vmap

# Computation of the roots
roots = polynomial_root_calculation_4th_degree_ferrari(a0,a1,a2,a3,a4) # (batch_size,4)

print("roots = \n", roots)  # list of tensor (for complex numbers)


a0 = 
 tensor([[ 1.3118e+03],
        [ 4.5853e+00],
        [ 2.1543e+00],
        [-8.1239e-02],
        [ 8.5578e+01]], dtype=torch.float64)
torch.Size([5, 1])
a torch.Size([5, 1])
x_cube  tensor([[[ 0.0294,  0.0000],
         [ 0.2448,  0.5083],
         [ 0.2448, -0.5083]],

        [[ 0.0364, -0.0000],
         [ 0.1192,  0.0000],
         [ 0.1192,  0.0000]],

        [[ 0.1275,  0.0000],
         [-0.2189,  0.2056],
         [-0.2189, -0.2056]],

        [[ 0.2673,  0.0000],
         [ 0.0172,  0.0000],
         [ 0.0172,  0.0000]],

        [[ 0.1784, -0.0000],
         [ 0.2803,  0.0000],
         [ 0.2803,  0.0000]]])
torch.Size([5, 3, 2])
x_cube_real = 
 tensor([[ 0.0294,  0.2448,  0.2448],
        [ 0.0364,  0.1192,  0.1192],
        [ 0.1275, -0.2189, -0.2189],
        [ 0.2673,  0.0172,  0.0172],
        [ 0.1784,  0.2803,  0.2803]])
torch.Size([5, 3])
a.shape =  torch.Size([5, 2])
b.shape =  torch.Size([5, 1])
frac = 
 tensor([[0.5642, 0.0000],
        [0.0041, 0.0000],

10. For each solution : computation of the camera position and rotation matrix

In [146]:
# For each solution of the polynomial
for i in range(4):
  print("ITERATIIIIIOOOOOOOOON = ", i)
  # Computation of trigonometrics forms
  cos_teta = torch.tensor(roots[i,:,0]).unsqueeze(-1) # real part of the root (batch_size,1)
  sin_teta = torch.sqrt(1-cos_teta**2)

  cot_alpha = ((phi1/phi2)*p1 + cos_teta*p2 -d12*b )/ ((phi1/phi2)*cos_teta* p2 - p1 + d12)

  sin_alpha = torch.sqrt(1/(cot_alpha**2+1))
  cos_alpha = torch.sqrt(1-sin_alpha**2)
  cos_alpha = torch.where(cot_alpha < 0, -cos_alpha, cos_alpha)  

  # Computation of the intermediate rotation's matrixs
  C_estimate = torch.stack([d12*cos_alpha*(sin_alpha*b + cos_alpha), d12*sin_alpha*cos_teta*(sin_alpha*b+cos_alpha), d12*sin_alpha*sin_teta*(sin_alpha*b+cos_alpha)],dim=1)  # (batch_size,3,1)
  # (batch_size,3,1)
  print("C_estimate = \n", C_estimate)  # Print C_estimate to check the values

  Q_row1 = torch.stack([-cos_alpha, -sin_alpha*cos_teta, -sin_alpha*sin_teta], dim=-1)  
  print("Q_row1 = \n", Q_row1)  # Print Q_row1 to check the values
  Q_row2 = torch.stack([sin_alpha, -cos_alpha*cos_teta, -cos_alpha*sin_teta],dim=-1)
  Q_row3 = torch.stack([0*sin_teta, -sin_teta, cos_teta], dim=-1)
  Q = torch.stack([Q_row1,Q_row2 ,Q_row3],dim=1).squeeze(2)  # (batch_size,3*3)
  print("Q = \n", Q)  # Print Q to check the values
  print(Q.shape)  # (batch_size,3,3)


  # Computation of the absolute camera center
  C_estimate = P1.unsqueeze(-1) + torch.matmul(torch.transpose(N, 1,2), C_estimate) # (batch_size,3,1)
  print("C_estimate = \n", C_estimate) 
  print(C_estimate.shape)  
  

  # Computation of the orientation matrix
  print("N transpo ",torch.transpose(N,1,2))
  print("Q transpo ",torch.transpose(Q, 1,2))

  print("produit 1 ",torch.matmul(torch.transpose(N,1,2),torch.transpose(Q, 1,2)))

  
  R_estimate = torch.matmul(torch.matmul(torch.transpose(N,1,2),torch.transpose(Q, 1,2)),T)   # (batch_size,3,3)
  print("R_estimate = \n", R_estimate)
  print(R_estimate.shape)  # (3,3)
  
  # Adding C and R to the solutions
  solutions[:,i,:,:1]= C_estimate
  solutions[:,i,:,1:] = R_estimate

print("solutions = \n", solutions)


ITERATIIIIIOOOOOOOOON =  0
C_estimate = 
 tensor([[[-2.6746],
         [ 0.5013],
         [ 1.6022]],

        [[ 1.6874],
         [ 4.4020],
         [ 4.6939]],

        [[ 4.5381],
         [ 1.6719],
         [ 5.3105]],

        [[ 2.6574],
         [ 1.3266],
         [ 5.9617]],

        [[-0.2754],
         [ 5.2347],
         [ 5.4889]]], dtype=torch.float64)
Q_row1 = 
 tensor([[[ 0.8470, -0.1587, -0.5074]],

        [[-0.2536, -0.6617, -0.7056]],

        [[-0.6318, -0.2328, -0.7393]],

        [[-0.3990, -0.1992, -0.8951]],

        [[ 0.0363, -0.6897, -0.7232]]], dtype=torch.float64)
Q = 
 tensor([[[ 0.8470, -0.1587, -0.5074],
         [ 0.5316,  0.2529,  0.8083],
         [ 0.0000, -0.9544,  0.2986]],

        [[-0.2536, -0.6617, -0.7056],
         [ 0.9673, -0.1735, -0.1850],
         [ 0.0000, -0.7294,  0.6841]],

        [[-0.6318, -0.2328, -0.7393],
         [ 0.7751, -0.1897, -0.6026],
         [ 0.0000, -0.9538,  0.3003]],

        [[-0.3990, -0.1992, -0.8951],
   

C:\Users\pmamalet\AppData\Local\Temp\ipykernel_11296\2422913758.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cos_teta = torch.tensor(roots[i,:,0]).unsqueeze(-1) # real part of the root (batch_size,1)


In [147]:
print("solutions = \n", solutions)
print(solutions.shape)  # (4,3,4)

solutions = 
 tensor([[[[ 1.5435e+00,  7.8278e-01, -7.7329e-02, -6.1748e-01],
          [ 2.7989e+00, -5.7610e-01, -4.6523e-01, -6.7206e-01],
          [ 1.1718e-01, -2.3530e-01,  8.8181e-01, -4.0872e-01]],

         [[ 1.5435e+00,  7.8278e-01, -7.7329e-02, -6.1748e-01],
          [ 2.7989e+00, -5.7610e-01, -4.6523e-01, -6.7206e-01],
          [ 1.1718e-01, -2.3530e-01,  8.8181e-01, -4.0872e-01]],

         [[-1.7218e-08,  1.0000e+00, -4.4421e-09,  1.5312e-09],
          [ 2.2907e-07, -4.4421e-09, -1.0000e+00, -3.0599e-08],
          [ 6.0000e+00,  1.5312e-09,  3.0599e-08, -1.0000e+00]],

         [[-5.4289e+00,  3.4614e-01, -5.0469e-01,  7.9087e-01],
          [ 2.9974e+00,  9.3295e-01,  2.7407e-01, -2.3343e-01],
          [-4.1881e+00, -9.8943e-02,  8.1864e-01,  5.6572e-01]]],


        [[[-3.8315e+00,  7.5655e-01, -2.7391e-02,  6.5336e-01],
          [ 1.4817e+00,  1.9570e-01, -9.4385e-01, -2.6617e-01],
          [ 3.6261e+00,  6.2397e-01,  3.2923e-01, -7.0871e-01]],

         [[ 1.

11. Reprojection of points in 2D from the newly estimated matrices to verify the estimation error.

In [160]:
def projection3D2D(point3D,C,R,A) :
  print("DASN PROJ 3D 2D ")
  print("C = \n", C)  # Print C to check the values
  print(C.shape)  # (3,1)
  print("R = \n", R)  # Print point3D to check the values
  print(R.shape)  # (3,)


  # 3D point = [ Xw, Yw, Zw ]'   (1*3)
  # T : camera translation matrix : (3*1)
  # R : camera rotation matrix : (3*3)
  # A : intraseca matrix of the camera : (3*3)
  # Output : return the coordonates of the point in 2D 

  PI = torch.cat((torch.eye(3, dtype=torch.float64),torch.zeros((3,1), dtype=torch.float64)),dim=1)  # (3*4)

  Rt = torch.cat((R,C),dim=1)               # (3*4)
  Rt = torch.cat((Rt,torch.tensor([[0,0,0,1]], dtype=torch.float64)),dim=0)   # (4*4)
  print("Rt = \n", Rt)  # Print Rt to check the values
  print(Rt.shape)  

  point3D_bis = torch.cat((torch.reshape(point3D,(3,1)),torch.tensor([[1]],dtype=torch.float64)),dim=0)   #(4*1)
  
  point2D = torch.tensordot(torch.tensordot(torch.tensordot(A,PI,dims=1),Rt,dims=1),point3D_bis,dims=1)  # 2D point = [u, v, w] (3*1)
  point2D = point2D / point2D[2]        # 2D point = [u, v, 1] (3*1)
  return point2D[:2]

print("C = \n", C)  # Print C to check the values
print(C.shape)  # (batch_size,3)
C_transpose = C.unsqueeze(-1) # (batch_size,3,1
print("C_transpose = \n", C_transpose)  # Print C_transpose to check the values
print(C_transpose.shape)  # (1*3)

print("P1 = \n", P1)  # Print P1 to check the values
print(P1.shape)  # (batch_size,3)

print("R = \n", R)  # Print R to check the values
print(R.shape)  # (batch_size,3,3)

print("A = \n", A)  # Print A to check the values
print(A.shape)  # (batch_size,3,3)

p1 = vmap(projection3D2D)(P1,C_transpose,R,A)
print("p1 = ", p1)
print(p1.shape)  # (2,1)
p2 = vmap(projection3D2D)(points3D[1],C_transpose,R,A)
print("p2 = ", p2)
p3 = vmap(projection3D2D)(points3D[2],C_transpose,R,A)
print("p3 = ", p3)
p4 = vmap(projection3D2D)(P4,C_transpose,R,A)
print("p4 = ", p4)




C = 
 tensor([[0., 0., 6.],
        [0., 0., 6.],
        [0., 0., 6.],
        [0., 0., 6.],
        [0., 0., 6.]], dtype=torch.float64)
torch.Size([5, 3])
C_transpose = 
 tensor([[[0.],
         [0.],
         [6.]],

        [[0.],
         [0.],
         [6.]],

        [[0.],
         [0.],
         [6.]],

        [[0.],
         [0.],
         [6.]],

        [[0.],
         [0.],
         [6.]]], dtype=torch.float64)
torch.Size([5, 3, 1])
P1 = 
 tensor([[-0.5207,  1.0252, -1.4844],
        [ 1.6953, -1.4334,  1.3427],
        [-0.5690, -0.9334, -0.7194],
        [-0.0642,  1.7793, -0.4778],
        [ 0.3497, -1.3591, -1.2870]], dtype=torch.float64)
torch.Size([5, 3])
R = 
 tensor([[[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.]],

        [[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.]],

        [[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.]],

        [[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0.

12. Calculation of errors = distance between the 2D points estimated from the found rotation and position matrices and the 2D points from the initial matrices

In [192]:
def distance(pt, pt_estimation):
    erreur = torch.tensor(0, dtype=torch.float64)  # Initialize error as a tensor
    for i in range(len(pt)): 
      erreur = erreur + torch.tensor((pt[i] - pt_estimation[i])**2, dtype=torch.float64)  # Ensure each term is a tensor
      #erreur += (pt[i] - pt_estimation[i])**2
    return torch.sqrt(erreur)


def affichage_erreur(solutions,points2D,points3D,A) : 
   # Compute the error of estimation for each points after the P3P algorithm 

   # solutions : solution matrix returned by P3P (4*3*4)
   # points 3D : 4 pts 3D used for P3P 
   # points 2D : 4 pts 2D used for P3P (image of the 3D points)
   
   P1 = points3D[0]
   P2 = points3D[1]
   P3 = points3D[2]
   P4 = points3D[3]

   erreurs = torch.empty((solutions.shape[1],), dtype=torch.float64)  # Initialize errors tensor to store the errors for each solution
   nb_sol = 0

   for i in range(solutions.shape[1]):  # Iterate over the solutions: 
      R = solutions[:,i,:,1:] 
      C = solutions[:,i,:,:1]
    
      if not torch.all(R==torch.zeros((3,3))) : 
        nb_sol += 1 
        print("------------ Solution n° : ",nb_sol,"----------------")
        print("R = \n",R,)
        print("T = \n",C,)

        p1_P3P = torch.reshape(vmap(projection3D2D)(P1,C,R,A),(batch_size,1,2))
        print("p1_P3P = \n", p1_P3P)  # Print p1_P3P to check the values
        p2_P3P = torch.reshape(vmap(projection3D2D)(P2,C,R,A),(batch_size,1,2))
        p3_P3P = torch.reshape(vmap(projection3D2D)(P3,C,R,A),(batch_size,1,2))
        p4_P3P = torch.reshape(vmap(projection3D2D)(P4,C,R,A),(batch_size,1,2))
        pt_2D_P3P = [p1_P3P,p2_P3P,p3_P3P,p4_P3P]   # (4,2)


        erreurs[i] = torch.tensor(0, dtype=torch.float64)
        for j in range(4) :
            erreur_pt = distance(points2D[:][j],pt_2D_P3P[:][j])
            print("Erreur pour le point ",j+1," = ",erreur_pt)
            erreurs[i]+=erreur_pt
        
   indice_min = 0
   min = erreurs[0]
   for i in range(1,len(erreurs)) :
    if erreurs[i]<min :
      min = erreurs[i]
      indice_min = i

   R_opti = solutions[:,indice_min,:,1:] 
   C_opti = solutions[:,indice_min,:,:1]
   print("\n------------ Best solution : ----------------")
   print("Solution n° :",indice_min+1,"\n")
   print("R estimé = \n", R_opti,"\n")
   print("T estimé = \n", C_opti, "\n")

affichage_erreur(solutions, [p1, p2, p3, p4], [P1,P2,P3, P4], A)

------------ Solution n° :  1 ----------------
R = 
 tensor([[[ 0.7828, -0.0773, -0.6175],
         [-0.5761, -0.4652, -0.6721],
         [-0.2353,  0.8818, -0.4087]],

        [[ 0.7565, -0.0274,  0.6534],
         [ 0.1957, -0.9439, -0.2662],
         [ 0.6240,  0.3292, -0.7087]],

        [[-0.0381, -0.3270, -0.9443],
         [ 0.7769,  0.5846, -0.2338],
         [ 0.6285, -0.7425,  0.2317]],

        [[ 0.9806, -0.0316, -0.1934],
         [-0.0288, -0.9994,  0.0172],
         [-0.1938, -0.0113, -0.9810]],

        [[ 0.5969,  0.2489, -0.7628],
         [-0.7074, -0.2853, -0.6467],
         [-0.3786,  0.9255,  0.0058]]], dtype=torch.float64)
T = 
 tensor([[[ 1.5435],
         [ 2.7989],
         [ 0.1172]],

        [[-3.8315],
         [ 1.4817],
         [ 3.6261]],

        [[ 6.4245],
         [ 0.6148],
         [-1.2539]],

        [[ 1.1846],
         [-0.0960],
         [ 5.7902]],

        [[ 5.4750],
         [ 4.1100],
         [-2.4811]]], dtype=torch.float64)
DASN PROJ

C:\Users\pmamalet\AppData\Local\Temp\ipykernel_11296\3685586606.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  erreur = erreur + torch.tensor((pt[i] - pt_estimation[i])**2, dtype=torch.float64)  # Ensure each term is a tensor


RuntimeError: output with shape [] doesn't match the broadcast shape [2, 2]